In [1]:
cd ../..

/Users/hanyu/Desktop/TextAttack


In [2]:
from textattack.transformations import WordSwap
import transformers
import string

In [3]:
#attack example
import os
import pandas as pd
import datasets
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper
tokenizer = transformers.AutoTokenizer.from_pretrained("Raychanan/bert-base-chinese-FineTuned-Binary-Best")
model = transformers.AutoModelForSequenceClassification.from_pretrained("Raychanan/bert-base-chinese-FineTuned-Binary-Best")

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

from textattack.goal_functions import UntargetedClassification
goal_function = UntargetedClassification(model_wrapper, query_budget=10000)

from textattack.datasets import HuggingFaceDataset

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [4]:
#get demo dataset path
path = os.path.abspath('')

path_list = path.split(os.sep)
path_list.append('examples/dataset/experiment_dataset.tsv')
data_path = os.path.join("/", *path_list)

dataset = datasets.load_dataset('csv', data_files=data_path, delimiter="\t")["train"]

dataset = HuggingFaceDataset(
        dataset,
#         lang="zh",
        dataset_columns=(["text"], "label"),
        label_names=["Negative", "Positive"]
    )

Using custom data configuration default-400a3aa4ae3a79f8
Reusing dataset csv (/Users/hanyu/.cache/huggingface/datasets/csv/default-400a3aa4ae3a79f8/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import ChineseWordSwapHowNet
from textattack.transformations import ChineseHomophoneCharacterSwap
from textattack.transformations import ChineseMorphonymCharacterSwap
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack import Attack

stopwords = set(
        ["、", "。", "〈", "〉", "《", "》", "一", "一个", "一些", "一何", "一切", "一则", "一方面", "一旦", "一来", "一样", "一种", "一般", "一转眼", "七", "万一", "三", "上", "上下", "下", "不", "不仅", "不但", "不光", "不单", "不只", "不外乎", "不如", "不妨", "不尽", "不尽然", "不得", "不怕", "不惟", "不成", "不拘", "不料", "不是", "不比", "不然", "不特", "不独", "不管", "不至于", "不若", "不论", "不过", "不问", "与", "与其", "与其说", "与否", "与此同时", "且", "且不说", "且说", "两者", "个", "个别", "中", "临", "为", "为了", "为什么", "为何", "为止", "为此", "为着", "乃", "乃至", "乃至于", "么", "之", "之一", "之所以", "之类", "乌乎", "乎", "乘", "九", "也", "也好", "也罢", "了", "二", "二来", "于", "于是", "于是乎", "云云", "云尔", "五", "些", "亦", "人", "人们", "人家", "什", "什么", "什么样", "今", "介于", "仍", "仍旧", "从", "从此", "从而", "他", "他人", "他们", "他们们", "以", "以上", "以为", "以便", "以免", "以及", "以故", "以期", "以来", "以至", "以至于", "以致", "们", "任", "任何", "任凭", "会", "似的", "但", "但凡", "但是", "何", "何以", "何况", "何处", "何时", "余外", "作为", "你", "你们", "使", "使得", "例如", "依", "依据", "依照", "便于", "俺", "俺们", "倘", "倘使", "倘或", "倘然", "倘若", "借", "借傥然", "假使", "假如", "假若", "做", "像", "儿", "先不先", "光", "光是", "全体", "全部", "八", "六", "兮", "共", "关于", "关于具体地说", "其", "其一", "其中", "其二", "其他", "其余", "其它", "其次", "具体地说", "具体说来", "兼之", "内", "再", "再其次", "再则", "再有", "再者", "再者说", "再说", "冒", "冲", "况且", "几", "几时", "凡", "凡是", "凭", "凭借", "出于", "出来", "分", "分别", "则", "则甚", "别", "别人", "别处", "别是", "别的", "别管", "别说", "到", "前后", "前此", "前者", "加之", "加以", "区", "即", "即令", "即使", "即便", "即如", "即或", "即若", "却", "去", "又", "又及", "及", "及其", "及至", "反之", "反而", "反过来", "反过来说", "受到", "另", "另一方面", "另外", "另悉", "只", "只当", "只怕", "只是", "只有", "只消", "只要", "只限", "叫", "叮咚", "可", "可以", "可是", "可见", "各", "各个", "各位", "各种", "各自", "同", "同时", "后", "后者", "向", "向使", "向着", "吓", "吗", "否则", "吧", "吧哒", "含", "吱", "呀", "呃", "呕", "呗", "呜", "呜呼", "呢", "呵", "呵呵", "呸", "呼哧", "咋", "和", "咚", "咦", "咧", "咱", "咱们", "咳", "哇", "哈", "哈哈", "哉", "哎", "哎呀", "哎哟", "哗", "哟", "哦", "哩", "哪", "哪个", "哪些", "哪儿", "哪天", "哪年", "哪怕", "哪样", "哪边", "哪里", "哼", "哼唷", "唉", "唯有", "啊", "啐", "啥", "啦", "啪达", "啷当", "喂", "喏", "喔唷", "喽", "嗡", "嗡嗡", "嗬", "嗯", "嗳", "嘎", "嘎登", "嘘", "嘛", "嘻", "嘿", "嘿嘿", "四", "因", "因为", "因了", "因此", "因着", "因而", "固然", "在", "在下", "在于", "地", "基于", "处在", "多", "多么", "多少", "大", "大家", "她", "她们", "好", "如", "如上", "如上所述", "如下", "如何", "如其", "如同", "如是", "如果", "如此", "如若", "始而", "孰料", "孰知", "宁", "宁可", "宁愿", "宁肯", "它", "它们", "对", "对于", "对待", "对方", "对比", "将", "小", "尔", "尔后", "尔尔", "尚且", "就", "就是", "就是了", "就是说", "就算", "就要", "尽", "尽管", "尽管如此", "岂但", "己", "已", "已矣", "巴", "巴巴", "年", "并", "并且", "庶乎", "庶几", "开外", "开始", "归", "归齐", "当", "当地", "当然", "当着", "彼", "彼时", "彼此", "往", "待", "很", "得", "得了", "怎", "怎么", "怎么办", "怎么样", "怎奈", "怎样", "总之", "总的来看", "总的来说", "总的说来", "总而言之", "恰恰相反", "您", "惟其", "慢说", "我", "我们", "或", "或则", "或是", "或曰", "或者", "截至", "所", "所以", "所在", "所幸", "所有", "才", "才能", "打", "打从", "把", "抑或", "拿", "按", "按照", "换句话说", "换言之", "据", "据此", "接着", "故", "故此", "故而", "旁人", "无", "无宁", "无论", "既", "既往", "既是", "既然", "日", "时", "时候", "是", "是以", "是的", "更", "曾", "替", "替代", "最", "月", "有", "有些", "有关", "有及", "有时", "有的", "望", "朝", "朝着", "本", "本人", "本地", "本着", "本身", "来", "来着", "来自", "来说", "极了", "果然", "果真", "某", "某个", "某些", "某某", "根据", "欤", "正值", "正如", "正巧", "正是", "此", "此地", "此处", "此外", "此时", "此次", "此间", "毋宁", "每", "每当", "比", "比及", "比如", "比方", "没奈何", "沿", "沿着", "漫说", "点", "焉", "然则", "然后", "然而", "照", "照着", "犹且", "犹自", "甚且", "甚么", "甚或", "甚而", "甚至", "甚至于", "用", "用来", "由", "由于", "由是", "由此", "由此可见", "的", "的确", "的话", "直到", "相对而言", "省得", "看", "眨眼", "着", "着呢", "矣", "矣乎", "矣哉", "离", "秒", "称", "竟而", "第", "等", "等到", "等等", "简言之", "管", "类如", "紧接着", "纵", "纵令", "纵使", "纵然", "经", "经过", "结果", "给", "继之", "继后", "继而", "综上所述", "罢了", "者", "而", "而且", "而况", "而后", "而外", "而已", "而是", "而言", "能", "能否", "腾", "自", "自个儿", "自从", "自各儿", "自后", "自家", "自己", "自打", "自身", "至", "至于", "至今", "至若", "致", "般的", "若", "若夫", "若是", "若果", "若非", "莫不然", "莫如", "莫若", "虽", "虽则", "虽然", "虽说", "被", "要", "要不", "要不是", "要不然", "要么", "要是", "譬喻", "譬如", "让", "许多", "论", "设使", "设或", "设若", "诚如", "诚然", "该", "说", "说来", "请", "诸", "诸位", "诸如", "谁", "谁人", "谁料", "谁知", "贼死", "赖以", "赶", "起", "起见", "趁", "趁着", "越是", "距", "跟", "较", "较之", "边", "过", "还", "还是", "还有", "还要", "这", "这一来", "这个", "这么", "这么些", "这么样", "这么点儿", "这些", "这会儿", "这儿", "这就是说", "这时", "这样", "这次", "这般", "这边", "这里", "进而", "连", "连同", "逐步", "通过", "遵循", "遵照", "那", "那个", "那么", "那么些", "那么样", "那些", "那会儿", "那儿", "那时", "那样", "那般", "那边", "那里", "都", "鄙人", "鉴于", "针对", "阿", "除", "除了", "除外", "除开", "除此之外", "除非", "随", "随后", "随时", "随着", "难道说", "零", "非", "非但", "非徒", "非特", "非独", "靠", "顺", "顺着", "首先", "︿", "！", "＃", "＄", "％", "＆", "（", "）", "＊", "＋", "，", "０", "１", "２", "３", "４", "５", "６", "７", "８", "９", "：", "；", "＜", "＞", "？", "＠", "［", "］", "｛", "｜", "｝", "～", "￥"]
    )
stopwords = stopwords.union(set(string.punctuation))

constraints = [RepeatModification(),
               StopwordModification(stopwords = stopwords)]

search_method = GreedyWordSwapWIR()

In [6]:
from tqdm import tqdm
from textattack.loggers import CSVLogger
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

In [8]:
transformation=ChineseHomophoneCharacterSwap()
attack = Attack(goal_function, constraints, transformation, search_method)

attack_args = AttackArgs(num_examples=500)

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  ChineseHomophoneCharacterSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 




  0%|          | 1/500 [00:48<6:41:36, 48.29s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   0%|          | 1/500 [00:48<6:41:37, 48.29s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (97%)]] --> [[[FAILED]]]

哈哈，宝贝很不错的，无色差，质量特别好，价格便宜，发货快，满意





[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   0%|          | 2/500 [01:24<5:48:48, 42.02s/it]
[Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:   0%|          | 2/500 [01:24<5:48:48, 42.03s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (98%)]] --> [[[FAILED]]]

不错，售后服务非常给力，一直相信京东




...


[Succeeded / Failed / Skipped / Total] 186 / 133 / 21 / 340:  68%|██████▊   | 341/500 [4:30:08<2:05:57, 47.53s/it]
[Succeeded / Failed / Skipped / Total] 186 / 134 / 21 / 341:  68%|██████▊   | 341/500 [4:30:08<2:05:57, 47.53s/it]

--------------------------------------------- Result 341 ---------------------------------------------
[[Negative (95%)]] --> [[[FAILED]]]

酒店设施、环境很差，很不舒适，非常不满意！





[Succeeded / Failed / Skipped / Total] 186 / 134 / 21 / 341:  68%|██████▊   | 342/500 [4:30:31<2:04:58, 47.46s/it]
[Succeeded / Failed / Skipped / Total] 187 / 134 / 21 / 342:  68%|██████▊   | 342/500 [4:30:31<2:04:58, 47.46s/it]

--------------------------------------------- Result 342 ---------------------------------------------
[[Positive (91%)]] --> [[Negative (96%)]]

可存三百多个单词，方便英语的口语学习！电话薄可以添加许多联系人的信息，这等于加了一些内容呀！

可存三百多个单词，方便英语的口语学习！电话薄可以添加许多联系人的信息，这等于假了一些内容呀！




KeyboardInterrupt: 

In [ ]:
transformation=ChineseWordSwapHowNet()
attack = Attack(goal_function, constraints, transformation, search_method)

attack_args = AttackArgs(num_examples=200)

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()

Initializing OpenHowNet succeeded!
Initializing similarity calculation succeeded!
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  ChineseWordSwapHowNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 




  0%|          | 1/200 [02:39<8:47:52, 159.16s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   0%|          | 1/200 [02:39<8:47:52, 159.16s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (97%)]] --> [[[FAILED]]]

哈哈，宝贝很不错的，无色差，质量特别好，价格便宜，发货快，满意





[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   1%|          | 2/200 [04:20<7:09:17, 130.09s/it]
[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   1%|          | 2/200 [04:20<7:09:17, 130.09s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (73%)]]

不错，售后服务非常给力，一直相信京东

不大离儿，人工客服一百八十度一泻千里，一时一刻信亚马逊




...

--------------------------------------------- Result 87 ---------------------------------------------
[[Positive (65%)]] --> [[[SKIPPED]]]

裤子很舒服，很修身，但是开线很严重，明天就出长差也没有时间换了





[Succeeded / Failed / Skipped / Total] 27 / 51 / 9 / 87:  44%|████▍     | 88/200 [2:46:50<3:32:20, 113.76s/it]
[Succeeded / Failed / Skipped / Total] 27 / 52 / 9 / 88:  44%|████▍     | 88/200 [2:46:50<3:32:20, 113.76s/it]

--------------------------------------------- Result 88 ---------------------------------------------
[[Positive (96%)]] --> [[[FAILED]]]

物流很快，裤子 颜色很正，穿着也很舒服，朋友都说太值了，好的没得说。





[Succeeded / Failed / Skipped / Total] 27 / 52 / 9 / 88:  44%|████▍     | 89/200 [2:46:52<3:28:06, 112.49s/it]
[Succeeded / Failed / Skipped / Total] 27 / 52 / 10 / 89:  44%|████▍     | 89/200 [2:46:52<3:28:06, 112.49s/it]

--------------------------------------------- Result 89 ---------------------------------------------
[[Positive (81%)]] --> [[[SKIPPED]]]

收到裤子了。。。顺丰的速递的速度还是可以的。。。不满意的是的裤子脱色




In [ ]:
from textattack.transformations import ChineseMorphonymCharacterSwap
transformation = ChineseMorphonymCharacterSwap()
attack = Attack(goal_function, constraints, transformation, search_method)

attack_args = AttackArgs(num_examples=500)

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()

In [ ]:
from textattack.transformations import CompositeTransformation
transformation = CompositeTransformation([ChineseMorphonymCharacterSwap(),ChineseWordSwapHowNet(), ChineseHomophoneCharacterSwap()])
attack = Attack(goal_function, constraints, transformation, search_method)

attack_args = AttackArgs(num_examples=500)

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()
